## Sudoku Solver 

In [ ]:
from gurobipy import Model, GRB, quicksum

def solve_sudoku(grid):
    # Create a new model
    model = Model("Sudoku")

    # Decision variables: x[i][j][k] = 1 if number k+1 is placed in cell (i, j)
    x = model.addVars(9, 9, 9, vtype=GRB.BINARY, name="x")

    # Objective function: Minimize 0 (just a feasibility problem)
    model.setObjective(0, GRB.MINIMIZE)

    # Constraint 1: Each number appears exactly once in each row
    for j in range(9):
        for k in range(9):
            model.addConstr(quicksum(x[i, j, k] for i in range(9)) == 1)

    # Constraint 2: Each number appears exactly once in each column
    for i in range(9):
        for k in range(9):
            model.addConstr(quicksum(x[i, j, k] for j in range(9)) == 1)

    # Constraint 3: Each cell contains exactly one number
    for i in range(9):
        for j in range(9):
            model.addConstr(quicksum(x[i, j, k] for k in range(9)) == 1)

    # Constraint 4: Pre-filled cells must match their values
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:  # If the cell is pre-filled
                model.addConstr(x[i, j, grid[i][j] - 1] == 1)

    # Constraint 5: Each number appears exactly once in each 3x3 subgrid
    for box_i in range(3):
        for box_j in range(3):
            for k in range(9):
                model.addConstr(
                    quicksum(
                        x[i, j, k]
                        for i in range(box_i * 3, (box_i + 1) * 3)
                        for j in range(box_j * 3, (box_j + 1) * 3)
                    )
                    == 1
                )

    # Solve the model
    model.optimize()

    # Extract the solution
    if model.status == GRB.OPTIMAL:
        solution = [[0 for _ in range(9)] for _ in range(9)]
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if x[i, j, k].x > 0.5:
                        solution[i][j] = k + 1
        return solution
    else:
        print("No solution found.")
        return None

In [9]:
# Example Sudoku grid (0 represents empty cells)
grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solution = solve_sudoku(grid)

# Print the solution
if solution:
    print("Sudoku Solution:")
    for row in solution:
        print(row)


Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-29
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Xeon(R) W-2245 CPU @ 3.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 354 rows, 729 columns and 2946 nonzeros
Model fingerprint: 0x39624697
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 354 rows and 729 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.00000000000

In [18]:
from pulp import LpProblem, LpVariable, lpSum, LpInteger, LpMinimize, LpStatus

def solve_sudoku_open_solver(grid):
    # Create the problem
    prob = LpProblem("Sudoku_Solver", LpMinimize)

    # Decision variables: x[i][j][k] = 1 if number k+1 is placed in cell (i, j)
    x = LpVariable.dicts("x", (range(9), range(9), range(9)), 0, 1, LpInteger)

    # Constraint 1: Each number appears exactly once in each row
    for j in range(9):
        for k in range(9):
            prob += lpSum(x[i][j][k] for i in range(9)) == 1

    # Constraint 2: Each number appears exactly once in each column
    for i in range(9):
        for k in range(9):
            prob += lpSum(x[i][j][k] for j in range(9)) == 1

    # Constraint 3: Each cell contains exactly one number
    for i in range(9):
        for j in range(9):
            prob += lpSum(x[i][j][k] for k in range(9)) == 1

    # Constraint 4: Pre-filled cells must match their values
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:  # If the cell is pre-filled
                prob += x[i][j][grid[i][j] - 1] == 1

    # Constraint 5: Each number appears exactly once in each 3x3 subgrid
    for box_i in range(3):
        for box_j in range(3):
            for k in range(9):
                prob += lpSum(
                    x[i][j][k]
                    for i in range(box_i * 3, (box_i + 1) * 3)
                    for j in range(box_j * 3, (box_j + 1) * 3)
                ) == 1

    # Solve the problem
    prob.solve()

    # Extract the solution
    if LpStatus[prob.status] == "Optimal":
        solution = [[0 for _ in range(9)] for _ in range(9)]
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if x[i][j][k].varValue == 1:
                        solution[i][j] = k + 1
        return solution
    else:
        print("No solution found.")
        return None


# Example Sudoku grid (0 represents empty cells)
grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solution = solve_sudoku_open_solver(grid)

# Print the solution
if solution:
    print("Sudoku Solution:")
    for row in solution:
        print(row)

Sudoku Solution:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


In [20]:
from ortools.sat.python import cp_model

def solve_sudoku_or_tools(grid):
    # Create the model
    model = cp_model.CpModel()

    # Create variables: x[i][j][k] is True if number k+1 is in cell (i, j)
    x = [[[model.NewBoolVar(f'x[{i}][{j}][{k}]') for k in range(9)] for j in range(9)] for i in range(9)]

    # Constraint 1: Each cell contains exactly one number
    for i in range(9):
        for j in range(9):
            model.AddExactlyOne(x[i][j][k] for k in range(9))

    # Constraint 2: Each number appears exactly once in each row
    for i in range(9):
        for k in range(9):
            model.AddExactlyOne(x[i][j][k] for j in range(9))

    # Constraint 3: Each number appears exactly once in each column
    for j in range(9):
        for k in range(9):
            model.AddExactlyOne(x[i][j][k] for i in range(9))

    # Constraint 4: Each number appears exactly once in each 3x3 subgrid
    for box_i in range(3):
        for box_j in range(3):
            for k in range(9):
                model.AddExactlyOne(
                    x[i][j][k]
                    for i in range(box_i * 3, (box_i + 1) * 3)
                    for j in range(box_j * 3, (box_j + 1) * 3)
                )

    # Constraint 5: Pre-filled cells
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                model.Add(x[i][j][grid[i][j] - 1] == 1)

    # Create the solver and solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Extract the solution
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        solution = [[0 for _ in range(9)] for _ in range(9)]
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if solver.Value(x[i][j][k]):
                        solution[i][j] = k + 1
        return solution
    else:
        print("No solution found.")
        return None

# Example Sudoku grid (0 represents empty cells)
grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solution = solve_sudoku_or_tools(grid)

# Print the solution
if solution:
    print("Sudoku Solution:")
    for row in solution:
        print(row)


Sudoku Solution:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


In [28]:
import cvxpy as cp
import numpy as np

def solve_sudoku_cvxpy(grid):
    # Create variables: x[i, j, k] is 1 if number k+1 is in cell (i, j), else 0
    x = cp.Variable((9, 9, 9), boolean=True)

    # Constraints list
    constraints = []

    # Constraint 1: Each cell contains exactly one number
    for i in range(9):
        for j in range(9):
            constraints.append(cp.sum(x[i, j, :]) == 1)

    # Constraint 2: Each number appears exactly once in each row
    for i in range(9):
        for k in range(9):
            constraints.append(cp.sum(x[i, :, k]) == 1)

    # Constraint 3: Each number appears exactly once in each column
    for j in range(9):
        for k in range(9):
            constraints.append(cp.sum(x[:, j, k]) == 1)

    # Constraint 4: Each number appears exactly once in each 3x3 subgrid
    for box_i in range(3):
        for box_j in range(3):
            for k in range(9):
                constraints.append(
                    cp.sum(
                        x[box_i * 3:(box_i + 1) * 3, box_j * 3:(box_j + 1) * 3, k]
                    ) == 1
                )

    # Constraint 5: Pre-filled cells
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                constraints.append(x[i, j, grid[i][j] - 1] == 1)

    # Objective function (dummy, as we only need feasibility)
    objective = cp.Minimize(0)

    # Problem definition
    problem = cp.Problem(objective, constraints)

    # Solve the problem
    problem.solve(solver=cp.SCIPY)

    # Extract the solution
    if problem.status == cp.OPTIMAL:
        solution = np.zeros((9, 9), dtype=int)
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    if x[i, j, k].value > 0.5:
                        solution[i, j] = k + 1
        return solution
    else:
        print("No solution found.")
        return None

# Example Sudoku grid (0 represents empty cells)
grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solution = solve_sudoku_cvxpy(grid)

# Print the solution
if solution is not None:
    print("Sudoku Solution:")
    print(solution)

Sudoku Solution:
[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]
